# Prediction Using Ensamble Tree

Part 1 --Data Preperation
1. basic (Temp, Month, Weekday, Hour)
2. with previous temp (Temp, previous 3 hours temp, Month, Weekday, Hour)
3. with previous load (Temp, previous 3 hours load, Month, Weekday, Hour)

Part 2 --ANN
1. Use "basic" data in Part 1 as input ,and generate one hot code for feature "Month", "Weekday", "Hour"
2. Use "basic" data in Part 1 as input

Part 3--Random Forest
1. Use "basic" data in Part 1 as input
2. Use "with previous temp" data in Part 1 as input
3. Use "with previous load" data in Part 1 as input

Part 4--XGBoost
1. Use "basic" data in Part 1 as input
2. Use "with previous load" data in Part 1 as input

Note: Part 2-Part 4 and all the methods inside are independly. For each method, all the input features has been resampled to 24 points a day (hourly) and the output is the load at this hour. Always run Data Preperation cells first and then run method cells.

## Data Preperation

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import csv
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
import pytz
import math

In [2]:
################
#READ LOAD DATA#
################
# set timezone
local_tz = pytz.timezone('America/Denver')

# read energy load from csv file
energy_load_read = pd.DataFrame.from_csv('/Users/leikang/Equota/algorithm/6.csv')['value']

# convert energy load's index from timestamp to datetime format
energy_index = [datetime.fromtimestamp(x,tz=local_tz) for x in energy_load_read.index]

# reset the load's index
energy_load_raw = pd.DataFrame({'Value':list(energy_load_read)},index=energy_index)

# resample to hourly load
energy_load = energy_load_raw.resample("H").mean().fillna(method='ffill')

In [3]:
################
#READ TEMP DATA#
################
# import hourly temp
temp = pd.DataFrame.from_csv('/Users/leikang/Equota/algorithm/temp_hourly.csv')
temp = temp[['WetBulbFarenheit','Date','Time']]

# store temp in data, datetime in index
data = [value for value in temp['WetBulbFarenheit']]
index = []

for j in temp.iterrows():

    c = datetime(int(str(j[1].Date)[0:4]),int(str(j[1].Date)[4:6]),int(str(j[1].Date)[6:8]),int(math.floor(j[1].Time/100)),int(j[1].Time-(math.floor(j[1].Time/100)*100)))
    index.append(c.replace(tzinfo=local_tz))

ts_temp = pd.DataFrame({'Temp':data},index = index)
def filter_num(x):
    try:
        return np.float(x)
    except:
        return np.nan
ts_temp['Temp'] = ts_temp['Temp'].apply(filter_num)
hourly_temp = ts_temp.copy().resample("H").mean().fillna(method='ffill')

In [4]:
total = pd.concat([hourly_temp,energy_load],axis=1).dropna()

In [5]:
total["Month"] = total.index.map(lambda x: x.month)
total["Weekday"] = total.index.map(lambda x: x.isoweekday())
total["Hour"] = total.index.map(lambda x: x.hour)


In [6]:
total.head()

,Temp,Value,Month,Weekday,Hour
2012-01-01 00:00:00-07:00,20.666667,34.094725,1,7,0
2012-01-01 01:00:00-07:00,20.333333,34.168583,1,7,1
2012-01-01 02:00:00-07:00,20.000000,34.844692,1,7,2
2012-01-01 03:00:00-07:00,20.666667,35.956950,1,7,3
2012-01-01 04:00:00-07:00,19.000000,35.518508,1,7,4


## Data Preperation--with previous temp
Don't run this part if you don't want to add previous temperature

In [194]:
move = list(total["Temp"])[:-1]
move.insert(0,np.nan)
total["Temp1"]= move

move = list(total["Temp"])[:-2]
move.insert(0,np.nan)
move.insert(0,np.nan)
total["Temp2"]= move

move = list(total["Temp"])[:-3]
move.insert(0,np.nan)
move.insert(0,np.nan)
move.insert(0,np.nan)
total["Temp3"]= move

In [195]:
total.head()

,Temp,Value,Month,Weekday,Hour,Temp1,Temp2,Temp3
2012-01-01 00:00:00-07:00,20.666667,34.094725,1,7,0,NaN,NaN,NaN
2012-01-01 01:00:00-07:00,20.333333,34.168583,1,7,1,20.666667,NaN,NaN
2012-01-01 02:00:00-07:00,20.000000,34.844692,1,7,2,20.333333,20.666667,NaN
2012-01-01 03:00:00-07:00,20.666667,35.956950,1,7,3,20.000000,20.333333,20.666667
2012-01-01 04:00:00-07:00,19.000000,35.518508,1,7,4,20.666667,20.000000,20.333333


In [197]:
total = total.dropna()

## Data Preperation--with previous value
Don't run this part if you don't want to add previous value

In [219]:
move = list(total["Value"])[:-1]
move.insert(0,np.nan)
total["Value1"]= move

move = list(total["Value"])[:-2]
move.insert(0,np.nan)
move.insert(0,np.nan)
total["Value2"]= move

move = list(total["Value"])[:-3]
move.insert(0,np.nan)
move.insert(0,np.nan)
move.insert(0,np.nan)
total["Value3"]= move

In [220]:
total.head()

,Temp,Value,Month,Weekday,Hour,Value1,Value2,Value3
2012-01-01 00:00:00-07:00,20.666667,34.094725,1,7,0,NaN,NaN,NaN
2012-01-01 01:00:00-07:00,20.333333,34.168583,1,7,1,34.094725,NaN,NaN
2012-01-01 02:00:00-07:00,20.000000,34.844692,1,7,2,34.168583,34.094725,NaN
2012-01-01 03:00:00-07:00,20.666667,35.956950,1,7,3,34.844692,34.168583,34.094725
2012-01-01 04:00:00-07:00,19.000000,35.518508,1,7,4,35.956950,34.844692,34.168583


In [221]:
total=total.dropna()

## Function Preperation--calculate absolute error

In [7]:
def cal_error_for_list(pred,actual):
    error_abs=[]
    for i in range(len(pred)):
        error_abs.append(abs(pred[i]-actual[i]))
    actual_sum = np.array(actual).sum()
    error_sum = np.array(error_abs).sum()
    print "Mape Error = ",float(error_sum)/float(actual_sum)
    return error_sum,actual_sum

## ANN with one hot code (44 input)

In [8]:
# seperate data to 2 parts: feature/target and show the feature we are going to use
target = total[["Value"]]
feature = total.copy().drop("Value",axis=1)
feature.head()

,Temp,Month,Weekday,Hour
2012-01-01 00:00:00-07:00,20.666667,1,7,0
2012-01-01 01:00:00-07:00,20.333333,1,7,1
2012-01-01 02:00:00-07:00,20.000000,1,7,2
2012-01-01 03:00:00-07:00,20.666667,1,7,3
2012-01-01 04:00:00-07:00,19.000000,1,7,4


In [9]:
# prepare feature table with one hot code
f_one = feature.copy()

weekday_one_hot = pd.get_dummies(f_one['Weekday'])
weekday_one_hot = weekday_one_hot.set_index(f_one.index)

month_one_hot = pd.get_dummies(f_one['Month'])
month_one_hot = month_one_hot.set_index(f_one.index)

hour_one_hot = pd.get_dummies(f_one['Hour'])
hour_one_hot = hour_one_hot.set_index(f_one.index)

f_weekday = [feature,weekday_one_hot,month_one_hot,hour_one_hot]

table_join = pd.concat(f_weekday, axis=1)
table = table_join.drop(['Weekday','Month','Hour'],axis=1)

In [10]:
f_norm = (table-table.mean())/(table.max()-table.min())
f_norm.head()

,Temp,1,2,3,4,5,6,7,1,2,...,14,15,16,17,18,19,20,21,22,23
2012-01-01 00:00:00-07:00,-0.318265,-0.144224,-0.142174,-0.142174,-0.142174,-0.142174,-0.142174,0.855092,0.915243,-0.079289,...,-0.041695,-0.041695,-0.041695,-0.041695,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581
2012-01-01 01:00:00-07:00,-0.323473,-0.144224,-0.142174,-0.142174,-0.142174,-0.142174,-0.142174,0.855092,0.915243,-0.079289,...,-0.041695,-0.041695,-0.041695,-0.041695,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581
2012-01-01 02:00:00-07:00,-0.328681,-0.144224,-0.142174,-0.142174,-0.142174,-0.142174,-0.142174,0.855092,0.915243,-0.079289,...,-0.041695,-0.041695,-0.041695,-0.041695,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581
2012-01-01 03:00:00-07:00,-0.318265,-0.144224,-0.142174,-0.142174,-0.142174,-0.142174,-0.142174,0.855092,0.915243,-0.079289,...,-0.041695,-0.041695,-0.041695,-0.041695,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581
2012-01-01 04:00:00-07:00,-0.344306,-0.144224,-0.142174,-0.142174,-0.142174,-0.142174,-0.142174,0.855092,0.915243,-0.079289,...,-0.041695,-0.041695,-0.041695,-0.041695,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581,-0.041581


In [11]:
Y = target.values.reshape(1,-1)[0]
X = f_norm.values

In [13]:
#######
##ANN##
#######
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.datasets import SupervisedDataSet,UnsupervisedDataSet
from pybrain.structure import LinearLayer
ds = SupervisedDataSet(X.shape[1], 1)
for x,y in zip(X,Y):
    ds.addSample(x,y)
prediction_net = buildNetwork(X.shape[1],30, 1, outclass=LinearLayer,bias=True)
trainer = BackpropTrainer(prediction_net, ds,verbose=True)
trainer.trainEpochs(200)

Total error: 11.8149982793
Total error: 6.97572933577
Total error: 6.24057038293
Total error: 5.84331510835
Total error: 5.62686835303
Total error: 5.46166081791
Total error: 5.31579886511
Total error: 5.23893792867
Total error: 5.1224465245
Total error: 4.98817268895
Total error: 4.96041327976
Total error: 4.94877840454
Total error: 4.80123717498
Total error: 4.73762666972
Total error: 4.68940644807
Total error: 4.66998696727
Total error: 4.68267591982
Total error: 4.61688146715
Total error: 4.564925607
Total error: 4.51114508724
Total error: 4.48608834154
Total error: 4.48896274725
Total error: 4.45676928205
Total error: 4.45664926636
Total error: 4.41459788098
Total error: 4.38717655816
Total error: 4.39369447303
Total error: 4.34239101221
Total error: 4.33214510611
Total error: 4.33618452793
Total error: 4.33226169372
Total error: 4.2699596109
Total error: 4.24295617998
Total error: 4.24178604304
Total error: 4.26273376972
Total error: 4.23259650381
Total error: 4.2257898099
Total 

In [14]:
ts = UnsupervisedDataSet(X.shape[1],) 
for x in X:
    ts.addSample(x)
# create prediction result
pre = prediction_net.activateOnDataset(ts)
pred = [x[0] for x in pre]
cal_error_for_list(pred,Y)

Mape Error =  0.0614929546518


(16764.536247952143, 272625.31688181817)

## ANN without one hot code

In [15]:
target = total[["Value"]]
feature = total.copy().drop("Value",axis=1)

In [16]:
feature.head()

,Temp,Month,Weekday,Hour
2012-01-01 00:00:00-07:00,20.666667,1,7,0
2012-01-01 01:00:00-07:00,20.333333,1,7,1
2012-01-01 02:00:00-07:00,20.000000,1,7,2
2012-01-01 03:00:00-07:00,20.666667,1,7,3
2012-01-01 04:00:00-07:00,19.000000,1,7,4


In [17]:
Y = target.values.reshape(1,-1)[0]
X = feature.values

In [31]:
#########
## ANN ##
#########
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.datasets import SupervisedDataSet,UnsupervisedDataSet
from pybrain.structure import LinearLayer
ds = SupervisedDataSet(X.shape[1], 1)
for x,y in zip(X,Y):
    ds.addSample(x,y)
prediction_net = buildNetwork(X.shape[1],30, 1, outclass=LinearLayer,bias=True)
trainer = BackpropTrainer(prediction_net, ds,verbose=True)
trainer.trainEpochs(200)

Total error: 28.836988871
Total error: 17.6873790861
Total error: 14.3754542505
Total error: 11.9680220375
Total error: 10.7948119425
Total error: 10.283548256
Total error: 10.1117328904
Total error: 9.97919342809
Total error: 9.89356729327
Total error: 9.83773117962
Total error: 9.75380816459
Total error: 9.67364246292
Total error: 9.67189716773
Total error: 9.62024394997
Total error: 9.60320584459
Total error: 9.53271244821
Total error: 9.52140826259
Total error: 9.43365737241
Total error: 9.39908874766
Total error: 9.40983278427
Total error: 9.33347963587
Total error: 9.31698861852
Total error: 9.28907550793
Total error: 9.31452506257
Total error: 9.29254707929
Total error: 9.26875324797
Total error: 9.2666232592
Total error: 9.21122507442
Total error: 9.172148959
Total error: 9.26949469154
Total error: 9.17523275813
Total error: 9.13350042801
Total error: 9.19327159372
Total error: 9.13493205883
Total error: 9.1221265544
Total error: 9.08945899321
Total error: 9.1123836333
Total er

In [42]:
ts = UnsupervisedDataSet(X.shape[1],) 
for x in X:
    ts.addSample(x)
# create prediction result
pre = prediction_net.activateOnDataset(ts)
pred = [x[0] for x in pre]
cal_error_for_list(pred,Y)

Mape Error =  0.100056512379


(27277.938393521181, 272625.31688181817)

## Random Forest --Use only external feature --Long term prediction

In [18]:
target = total[["Value"]]
feature = total.copy().drop("Value",axis=1)
Y = target.values.reshape(1,-1)[0]
X = feature.values
feature.head()

,Temp,Month,Weekday,Hour
2012-01-01 00:00:00-07:00,20.666667,1,7,0
2012-01-01 01:00:00-07:00,20.333333,1,7,1
2012-01-01 02:00:00-07:00,20.000000,1,7,2
2012-01-01 03:00:00-07:00,20.666667,1,7,3
2012-01-01 04:00:00-07:00,19.000000,1,7,4


In [19]:
# cal insample error
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=500)
clf = clf.fit(X, Y)
pred = clf.predict(X)
cal_error_for_list(pred,Y)

Mape Error =  0.0261402123648


(7126.4836793184249, 272625.31688181817)

In [22]:
# see which feature is important
print "Importance of each feature\n"
for name,score in zip(feature.columns,clf.feature_importances_ ):
    print name,":",score

Importance of each feature

Temp : 0.11190713602
Month : 0.0662885915966
Weekday : 0.0749302472632
Hour : 0.74687402512


In [26]:
# cal outsample error using random sample points 
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
ss = cross_validation.ShuffleSplit(len(X), 20, 0.05, random_state=0)
error_list = []
actual_list = []

for train,test in ss:
    clf = RandomForestRegressor(n_estimators=500)
    clf = clf.fit(X[train], Y[train])
    pred = clf.predict(X[test])
    error,actual = cal_error_for_list(pred, Y[test])
    error_list.append(error)
    actual_list.append(actual)
error_sum = np.array(error_list).sum()
actual_sum = np.array(actual_list).sum()
print "Total Mape Error = ",float(error_sum)/float(actual_sum)

Mape Error =  0.0717077966746
Mape Error =  0.0647415960819
Mape Error =  0.0612288369631
Mape Error =  0.0636248082063
Mape Error =  0.0650705712583
Mape Error =  0.0565046076059
Mape Error =  0.0622906789766
Mape Error =  0.0633016517725
Mape Error =  0.0654073006919
Mape Error =  0.0627641681949
Mape Error =  0.0652783324614
Mape Error =  0.0699954394153
Mape Error =  0.0643254843765
Mape Error =  0.0603114759418
Mape Error =  0.0639881945207
Mape Error =  0.0621714289946
Mape Error =  0.0631888974869
Mape Error =  0.0677674198692
Mape Error =  0.0651724077782
Mape Error =  0.0648377164218
Total Mape Error =  0.0641790785639


In [28]:
# cal outsample error using continous points
from sklearn.cross_validation import KFold
kf = KFold(len(X), n_folds=365)
error_list = []
actual_list = []
plot_actual = []
plot_pred = []
for train,test in kf:
    clf = RandomForestRegressor(n_estimators=500)
    clf = clf.fit(X[train], Y[train])
    pred = clf.predict(X[test])
    error,actual = cal_error_for_list(pred, Y[test])
    error_list.append(error)
    actual_list.append(actual)
    plot_actual.append(Y[test])
    plot_pred.append(pred)
error_sum = np.array(error_list).sum()
actual_sum = np.array(actual_list).sum()
print "Total Mape Error = ",float(error_sum)/float(actual_sum)

Mape Error =  0.139098826615
Mape Error =  0.0724317147867
Mape Error =  0.0758353593595
Mape Error =  0.0726900700201
Mape Error =  0.0912937885787
Mape Error =  0.0562571560018
Mape Error =  0.118373579609
Mape Error =  0.0840728487358
Mape Error =  0.0920807845357
Mape Error =  0.0744075066721
Mape Error =  0.108653677901
Mape Error =  0.116015370494
Mape Error =  0.142004594652
Mape Error =  0.066850982222
Mape Error =  0.0693279880431
Mape Error =  0.0984780618535
Mape Error =  0.0704048728994
Mape Error =  0.0846681989182
Mape Error =  0.0998617338751
Mape Error =  0.172620587886
Mape Error =  0.0695368516721
Mape Error =  0.112938858044
Mape Error =  0.0926531927826
Mape Error =  0.0854512543978
Mape Error =  0.0701670925332
Mape Error =  0.117721703122
Mape Error =  0.0726105473723
Mape Error =  0.0568393493951
Mape Error =  0.0891405023948
Mape Error =  0.0946283383975
Mape Error =  0.0715361318754
Mape Error =  0.0567450432985
Mape Error =  0.0705306368491
Mape Error =  0.074

In [29]:
def to_list(x):
    l = []
    for i in x:
        for j in i:
            l.append(j)
    return l
plot_actual_list = to_list(plot_actual)
plot_pred_list = to_list(plot_pred)

In [30]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [31]:
p = figure(y_range = (0,70))
x = range(len(plot_pred_list))
p.line(x,plot_pred_list,color = "darkred",legend="Pred")
p.line(x,plot_actual_list,color = "navy",legend = "Real")
show(p)

## Random forest -- Use only external feature--with previous temp(temp delay)

In [198]:
target = total[["Value"]]
feature = total.copy().drop("Value",axis=1)
Y = target.values.reshape(1,-1)[0]
X = feature.values
feature.head()

,Temp,Month,Weekday,Hour,Temp1,Temp2,Temp3
2012-01-01 03:00:00-07:00,20.666667,1,7,3,20.000000,20.333333,20.666667
2012-01-01 04:00:00-07:00,19.000000,1,7,4,20.666667,20.000000,20.333333
2012-01-01 05:00:00-07:00,19.000000,1,7,5,19.000000,20.666667,20.000000
2012-01-01 06:00:00-07:00,19.000000,1,7,6,19.000000,19.000000,20.666667
2012-01-01 07:00:00-07:00,18.000000,1,7,7,19.000000,19.000000,19.000000


In [199]:
# cal insample error
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=500)
clf = clf.fit(X, Y)
pred = clf.predict(X)
cal_error_for_list(pred,Y)

Mape Error =  0.0228257114162


(6220.5132944480347, 272522.20888181822)

In [200]:
# see which feature is important
print "Importance of each feature\n"
for name,score in zip(feature.columns,clf.feature_importances_ ):
    print name,":",score

Importance of each feature

Temp : 0.0281300625286
Month : 0.0514156926606
Weekday : 0.0666751856797
Hour : 0.737671873761
Temp1 : 0.0229113175863
Temp2 : 0.0358535157394
Temp3 : 0.0573423520444


In [204]:
# cal outsample error using random sample
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
ss = cross_validation.ShuffleSplit(len(X), 20, 0.05, random_state=0)
error_list = []
actual_list = []

for train,test in ss:
    clf = RandomForestRegressor(n_estimators=500)
    clf = clf.fit(X[train], Y[train])
    pred = clf.predict(X[test])
    error,actual = cal_error_for_list(pred, Y[test])
    error_list.append(error)
    actual_list.append(actual)
error_sum = np.array(error_list).sum()
actual_sum = np.array(actual_list).sum()
print "Total Mape Error = ",float(error_sum)/float(actual_sum)

Mape Error =  0.0653710731873
Mape Error =  0.065494308852
Mape Error =  0.0564755932259
Mape Error =  0.061090966353
Mape Error =  0.0626029560243
Mape Error =  0.0621122838242
Mape Error =  0.0643621454073
Mape Error =  0.0647727491776
Mape Error =  0.0668049550268
Mape Error =  0.0651276089757
Mape Error =  0.0700038244298
Mape Error =  0.0605585514489
Mape Error =  0.0627088866794
Mape Error =  0.0649620582448
Mape Error =  0.0644099568484
Mape Error =  0.0666629245308
Mape Error =  0.0660694145119
Mape Error =  0.0604268492219
Mape Error =  0.0613949249805
Mape Error =  0.0550553914504
Total Mape Error =  0.0633362695565


## Random Forest --Use external feature and previous load--Short term prediction

In [222]:
target = total[["Value"]]
feature = total.copy().drop("Value",axis=1)
Y = target.values.reshape(1,-1)[0]
X = feature.values
feature.head()

,Temp,Month,Weekday,Hour,Value1,Value2,Value3
2012-01-01 03:00:00-07:00,20.666667,1,7,3,34.844692,34.168583,34.094725
2012-01-01 04:00:00-07:00,19.000000,1,7,4,35.956950,34.844692,34.168583
2012-01-01 05:00:00-07:00,19.000000,1,7,5,35.518508,35.956950,34.844692
2012-01-01 06:00:00-07:00,19.000000,1,7,6,36.074642,35.518508,35.956950
2012-01-01 07:00:00-07:00,18.000000,1,7,7,36.882300,36.074642,35.518508


In [224]:
# cal insample error
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=500)
clf = clf.fit(X, Y)
pred = clf.predict(X)
cal_error_for_list(pred,Y)

Mape Error =  0.0153981096801


(4196.3268626180297, 272522.20888181822)

In [225]:
# see which feature is important
print "Importance of each feature\n"
for name,score in zip(feature.columns,clf.feature_importances_ ):
    print name,":",score

Importance of each feature

Temp : 0.0108027255531
Month : 0.00518739526359
Weekday : 0.0063177631235
Hour : 0.0868868607124
Value1 : 0.865019489945
Value2 : 0.00968752381045
Value3 : 0.0160982415923


In [227]:
# cal outsample error using random sample
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
ss = cross_validation.ShuffleSplit(len(X), 20, 0.05, random_state=0)
error_list = []
actual_list = []

for train,test in ss:
    clf = RandomForestRegressor(n_estimators=500)
    clf = clf.fit(X[train], Y[train])
    pred = clf.predict(X[test])
    error,actual = cal_error_for_list(pred, Y[test])
    error_list.append(error)
    actual_list.append(actual)
error_sum = np.array(error_list).sum()
actual_sum = np.array(actual_list).sum()
print "Total Mape Error = ",float(error_sum)/float(actual_sum)

Mape Error =  0.040915902479
Mape Error =  0.0460398936047
Mape Error =  0.0359812001673
Mape Error =  0.0418314157013
Mape Error =  0.0396328837052
Mape Error =  0.0399420624362
Mape Error =  0.0432185209596
Mape Error =  0.0444632319805
Mape Error =  0.044300026159
Mape Error =  0.0427542528083
Mape Error =  0.0432768534563
Mape Error =  0.0387837125108
Mape Error =  0.0405968262609
Mape Error =  0.042157909402
Mape Error =  0.041058570442
Mape Error =  0.0434259163175
Mape Error =  0.0441071844921
Mape Error =  0.0443680265666
Mape Error =  0.0412951819576
Mape Error =  0.0401145399396
Total Mape Error =  0.0419307497552


In [111]:
def to_list(x):
    l = []
    for i in x:
        for j in i:
            l.append(j)
    return l
plot_actual_list = to_list(plot_actual)
plot_pred_list = to_list(plot_pred)

In [2]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [3]:
p = figure(y_range = (0,70))
x = range(len(plot_pred_list))
p.line(x,plot_pred_list,color = "darkred",legend="Predicted")
p.line(x,plot_actual_list,color = "navy",legend = "Actual")
show(p)

NameError: name 'plot_pred_list' is not defined

## XGBoost -- Only external feature

In [8]:
import xgboost as xgb

In [9]:
target = total[["Value"]]
feature = total.copy().drop("Value",axis=1)
Y = target.values.reshape(1,-1)[0]
X = feature.values
feature.head()

,Temp,Month,Weekday,Hour
2012-01-01 00:00:00-07:00,20.666667,1,7,0
2012-01-01 01:00:00-07:00,20.333333,1,7,1
2012-01-01 02:00:00-07:00,20.000000,1,7,2
2012-01-01 03:00:00-07:00,20.666667,1,7,3
2012-01-01 04:00:00-07:00,19.000000,1,7,4


In [47]:
# config

n_estimators = 1000

# encounter overfitting directly
max_depth = 5
min_child_weight = 0.6
gamma =2

# encounter overfitting by add randomness
subsample = 0.3
seed = 3


In [48]:
# insample error
xgb_model = xgb.XGBRegressor(
    n_estimators = n_estimators,
    max_depth=max_depth,
    min_child_weight = min_child_weight,
    gamma=gamma).fit(X,Y)
pred = xgb_model.predict(X)
cal_error_for_list(pred,Y)

Mape Error =  0.0543368650726


(14813.6050587886, 272625.31688181817)

In [49]:
# cal outsample error using random sample
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
ss = cross_validation.ShuffleSplit(len(X), 20, 0.05, random_state=0)
error_list = []
actual_list = []

for train,test in ss:
    xgb_model = xgb.XGBRegressor(
        n_estimators = n_estimators,
        max_depth=max_depth,
        min_child_weight = min_child_weight
        ,gamma=gamma).fit(X[train], Y[train])
    pred = xgb_model.predict(X[test])
    error,actual = cal_error_for_list(pred, Y[test])
    error_list.append(error)
    actual_list.append(actual)
error_sum = np.array(error_list).sum()
actual_sum = np.array(actual_list).sum()
print "Total Mape Error = ",float(error_sum)/float(actual_sum)

Mape Error =  0.0650880037751
Mape Error =  0.0609888912244
Mape Error =  0.0598203231417
Mape Error =  0.0634666884815
Mape Error =  0.0637521864607
Mape Error =  0.0568759718694
Mape Error =  0.0600150854171
Mape Error =  0.0633803327597
Mape Error =  0.0634513588236
Mape Error =  0.0606506840876
Mape Error =  0.0622841404703
Mape Error =  0.0686908701242
Mape Error =  0.0611256572046
Mape Error =  0.0610122688423
Mape Error =  0.0598990339714
Mape Error =  0.0584745593843
Mape Error =  0.0620511763262
Mape Error =  0.0645363096724
Mape Error =  0.0616451785077
Mape Error =  0.0615161762923
Total Mape Error =  0.0619338043205


## XGBoost -- External feature and previous load

In [174]:
target = total[["Value"]]
feature = total.copy().drop("Value",axis=1)
Y = target.values.reshape(1,-1)[0]
X = feature.values
feature.head()

,Temp,Month,Weekday,Hour,Value1,Value2,Value3
2012-01-01 03:00:00-07:00,20.666667,1,7,3,34.844692,34.168583,34.094725
2012-01-01 04:00:00-07:00,19.000000,1,7,4,35.956950,34.844692,34.168583
2012-01-01 05:00:00-07:00,19.000000,1,7,5,35.518508,35.956950,34.844692
2012-01-01 06:00:00-07:00,19.000000,1,7,6,36.074642,35.518508,35.956950
2012-01-01 07:00:00-07:00,18.000000,1,7,7,36.882300,36.074642,35.518508


In [186]:
# config
n_esti = 500

In [187]:
# insample error
xgb_model = xgb.XGBRegressor(n_estimators = n_esti).fit(X,Y)
pred = xgb_model.predict(X)
cal_error_for_list(pred,Y)

Mape Error =  0.0381150325048


(10387.192849820502, 272522.20888181822)

In [188]:
# cal outsample error using random sample
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
ss = cross_validation.ShuffleSplit(len(X), 20, 0.05, random_state=0)
error_list = []
actual_list = []

for train,test in ss:
    xgb_model = xgb.XGBRegressor(n_estimators = n_esti).fit(X[train], Y[train])
    pred = xgb_model.predict(X[test])
    error,actual = cal_error_for_list(pred, Y[test])
    error_list.append(error)
    actual_list.append(actual)
error_sum = np.array(error_list).sum()
actual_sum = np.array(actual_list).sum()
print "Total Mape Error = ",float(error_sum)/float(actual_sum)

Mape Error =  0.0447857075796
Mape Error =  0.0480430345307
Mape Error =  0.040337686451
Mape Error =  0.0457610253683
Mape Error =  0.0416110750089
Mape Error =  0.0441645621681
Mape Error =  0.0461445596733
Mape Error =  0.0437049336027
Mape Error =  0.0460379968929
Mape Error =  0.0461603609921
Mape Error =  0.0432624781897
Mape Error =  0.0406456802448
Mape Error =  0.0446351421007
Mape Error =  0.0453016585541
Mape Error =  0.0439552775158
Mape Error =  0.0457402659554
Mape Error =  0.0469054507789
Mape Error =  0.0492058019106
Mape Error =  0.0418586832465
Mape Error =  0.0414193823266
Total Mape Error =  0.0444928710633
